# Prepare Datasets for Benchmarking Tasks

> author: Shizhenkun   
> email: zhenkun.shi@tib.cas.cn   
> date: 2022-10-05  


## Dataset1. Enzyme None-enzyme Dataset
The enzyme dataset is consists of two parts: <u>a training set</u> and <u>a testing set</u>.   
The training set is from snapshot Feb-2018 and ***excludes*** those <u>deleted items</u> and <u>sequences changed items</u> in snapshot Feb-2022.    
The training set is consists of ***469,134*** records, of which ***222,567*** are enzymes, and ***246,567*** are none-enzymes.   
The testing set is from snapshot Feb-2022 and excludes these items that appeared in snapshot Feb-2018.   
The testing set is consists of ***10,614*** records, of which ***5111*** are enzymes, and ***5503*** are none-enzymes.   
Unlike previous works,  we did not filter any sequences in terms of length and homology to make the data more inclusive. We make a label for each sequence, 1 for enzyme and 0 for none-enzyme.   

## Dataset2. Enzyme Function Quantity Dataset
The enzyme quantity dataset only contains enzyme data, contain ***222,567*** records. The function quantity ranges from 1 to 8.

## Dataset 3: EC Dataset

The EC dataset consists of 227,678 enzyme records, 222,567 are training-set, and the rest 5111 are testing-set, covering 6,031 EC numbers. Up to Feb 2022, ***cmopared with [ExplorEnz](https://www.enzyme-database.org/stats.php) CURRENT EC = 6674***, there still exist 643 EC numbers that the model can not handle in the benchmarking. Thus, we exclude the sequences with these 267 EC numbers in the evaluation process. But, this problem can be resolved in the production scenario because we use the entire data from Swiss-Prot. Now the EC coverage is 6,031 and can be automatically extended, for the training is real-time based on the publication of Swiss-Prot every 8 weeks. 

## 1. Import packages

In [1]:
import numpy as np
import pandas as pd
import sys,os
from tqdm import tqdm
import config as cfg
from functools import reduce

from tools import filetool as ftool
from tools import exact_ec_from_uniprot as exactec
from tools import funclib
from tools import minitools as mtool

from tools import embdding_onehot as onehotebd

from pandarallel import pandarallel # 导入pandaralle
pandarallel.initialize(progress_bar=False)


%load_ext autoreload
%autoreload 2

INFO: Pandarallel will run on 52 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


## 2. Download rawdata from unisprot

> IF first time run pls uncomment the cell below, this will take longer time based on your internet speed

In [2]:
# # #snapshot 2018-02
# # ftool.wget(download_url=cfg.URL_SPROT_SNAP201802, save_file=cfg.FILE_SPROT_SNAP201802)

# #snapshot 2019-12
# ftool.wget(download_url=cfg.URL_SPROT_SNAP201902, save_file=cfg.FILE_SPROT_SNAP201902)

# # #snapshot 2020-06
# # ftool.wget(download_url=cfg.URL_SPROT_SNAP202006, save_file=cfg.FILE_SPROT_SNAP202006)

# #snapshot 2021-02
# ftool.wget(download_url=cfg.URL_SPROT_SNAP202102, save_file=cfg.FILE_SPROT_SNAP202102)

# # #snapshot 2022-02
# # ftool.wget(download_url=cfg.URL_SPROT_SNAP202202, save_file=cfg.FILE_SPROT_SNAP202202)

## 3. Extract records from rawdata

In [4]:
cmd_array = [
    # 2018 data
    f'tar -zxvf {cfg.FILE_SPROT_SNAP201802} -C {cfg.DIR_UNIPROT}',
    f'mv {cfg.DIR_UNIPROT}uniprot_sprot.dat.gz {cfg.DIR_UNIPROT}sprot2018.data.gz', 
    f'rm -f {cfg.DIR_UNIPROT}uniprot_sprot.fasta.gz {cfg.DIR_UNIPROT}uniprot_sprot_varsplic.fasta.gz {cfg.DIR_UNIPROT}uniprot_sprot.xml.gz',
    
    # 2019 data
    f'tar -zxvf {cfg.FILE_SPROT_SNAP201902} -C {cfg.DIR_UNIPROT}',
    f'mv {cfg.DIR_UNIPROT}uniprot_sprot.dat.gz {cfg.DIR_UNIPROT}sprot2019.data.gz', 
    f'rm -f {cfg.DIR_UNIPROT}uniprot_sprot.fasta.gz {cfg.DIR_UNIPROT}uniprot_sprot_varsplic.fasta.gz {cfg.DIR_UNIPROT}uniprot_sprot.xml.gz',

    # 2020 data
    f'tar -zxvf {cfg.FILE_SPROT_SNAP202006} -C {cfg.DIR_UNIPROT}',
    f'mv {cfg.DIR_UNIPROT}uniprot_sprot.dat.gz {cfg.DIR_UNIPROT}sprot2020.data.gz', 
    f'rm -f {cfg.DIR_UNIPROT}uniprot_sprot.fasta.gz {cfg.DIR_UNIPROT}uniprot_sprot_varsplic.fasta.gz {cfg.DIR_UNIPROT}uniprot_sprot.xml.gz',

    # 2021 data
    f'tar -zxvf {cfg.FILE_SPROT_SNAP202102} -C {cfg.DIR_UNIPROT}',
    f'mv {cfg.DIR_UNIPROT}uniprot_sprot.dat.gz {cfg.DIR_UNIPROT}sprot2021.data.gz', 
    f'rm -f {cfg.DIR_UNIPROT}uniprot_sprot.fasta.gz {cfg.DIR_UNIPROT}uniprot_sprot_varsplic.fasta.gz {cfg.DIR_UNIPROT}uniprot_sprot.xml.gz',

    # 2022 data
    f'tar -zxvf {cfg.FILE_SPROT_SNAP202202} -C {cfg.DIR_UNIPROT}',
    f'mv {cfg.DIR_UNIPROT}uniprot_sprot.dat.gz {cfg.DIR_UNIPROT}sprot2022.data.gz', 
    f'rm -f {cfg.DIR_UNIPROT}uniprot_sprot.fasta.gz {cfg.DIR_UNIPROT}uniprot_sprot_varsplic.fasta.gz {cfg.DIR_UNIPROT}uniprot_sprot.xml.gz'
]

[os.system(item) for item in cmd_array]

# snapshot 2018-02
exactec.run_exact_task(infile=f'{cfg.DIR_UNIPROT}sprot2018.data.gz', outfile=f'{cfg.DIR_UNIPROT}sprot2018.tsv')

# #snapshot 2019-02
exactec.run_exact_task(infile=f'{cfg.DIR_UNIPROT}sprot2019.data.gz', outfile=f'{cfg.DIR_UNIPROT}sprot2019.tsv')

#snapshot 2020-06
exactec.run_exact_task(infile=f'{cfg.DIR_UNIPROT}sprot2020.data.gz', outfile=f'{cfg.DIR_UNIPROT}sprot2020.tsv')

#snapshot 2021-06
exactec.run_exact_task(infile=f'{cfg.DIR_UNIPROT}sprot2021.data.gz', outfile=f'{cfg.DIR_UNIPROT}sprot2021.tsv')

# #snapshot 2022-02
exactec.run_exact_task(infile=f'{cfg.DIR_UNIPROT}sprot2022.data.gz', outfile=f'{cfg.DIR_UNIPROT}sprot2022.tsv')

## 4. Load records & Drop Duplicates

In [21]:
#加载数据并转换时间格式
sprot2018 = pd.read_csv(f'{cfg.DIR_UNIPROT}sprot2018.tsv', sep='\t',header=0) #读入文件
sprot2018 = mtool.convert_DF_dateTime(inputdf = sprot2018)

sprot2019 = pd.read_csv(f'{cfg.DIR_UNIPROT}sprot2019.tsv', sep='\t',header=0) #读入文件
sprot2019 = mtool.convert_DF_dateTime(inputdf = sprot2019)

sprot2020 = pd.read_csv(f'{cfg.DIR_UNIPROT}sprot2020.tsv', sep='\t',header=0) #读入文件
sprot2020 = mtool.convert_DF_dateTime(inputdf = sprot2020)

sprot2021 = pd.read_csv(f'{cfg.DIR_UNIPROT}sprot2021.tsv', sep='\t',header=0) #读入文件
sprot2021 = mtool.convert_DF_dateTime(inputdf = sprot2021)

sprot2022 = pd.read_csv(f'{cfg.DIR_UNIPROT}sprot2022.tsv', sep='\t',header=0) #读入文件
sprot2022 = mtool.convert_DF_dateTime(inputdf = sprot2022)


#Drop Duplicates
sprot2018.drop_duplicates(subset=['seq'], keep='first', inplace=True)
sprot2018.reset_index(drop=True, inplace=True)

sprot2019.drop_duplicates(subset=['seq'], keep='first', inplace=True)
sprot2019.reset_index(drop=True, inplace=True)

sprot2020.drop_duplicates(subset=['seq'], keep='first', inplace=True)
sprot2020.reset_index(drop=True, inplace=True)

sprot2021.drop_duplicates(subset=['seq'], keep='first', inplace=True)
sprot2021.reset_index(drop=True, inplace=True)

sprot2022.drop_duplicates(subset=['seq'], keep='first', inplace=True)
sprot2022.reset_index(drop=True, inplace=True)


In [22]:
sprot2018.head(3)

,id,name,isenzyme,isMultiFunctional,functionCounts,ec_number,ec_specific_level,date_integraged,date_sequence_update,date_annotation_update,seq,seqlength
0,Q3J1A3,LHB1_RHOS4,False,False,0,-,0,1986-07-21,2007-01-23,2017-10-25,MADKSDLGYTGLTDEQAQELHSVYMSGLWLFSAVAIVAHLAVYIWRPWF,49
1,P02157,MYG_MELME,False,False,0,-,0,1986-07-21,2007-01-23,2017-11-22,MGLSDGEWQLVLNVWGKVEADLAGHGQEVLIRLFKGHPETLEKFDK...,154
2,P02178,MYG_MEGNO,False,False,0,-,0,1986-07-21,2007-01-23,2017-11-22,MVLSDAEWQLVLNIWAKVEADVAGHGQDILIRLFKGHPETLEKFDK...,154


In [23]:
sprot2019.head(3)

,id,name,isenzyme,isMultiFunctional,functionCounts,ec_number,ec_specific_level,date_integraged,date_sequence_update,date_annotation_update,seq,seqlength
0,P03375,ENV_HV1B1,False,False,0,-,0,1986-07-21,1986-07-21,2019-01-16,MRVKEKYQHLWRWGWRWGTMLLGMLMICSATEKLWVTVYYGVPVWK...,856
1,P03356,POL_MLVAV,True,True,6,"3.4.23.-, 2.7.7.49, 2.7.7.7, 3.1.26.4, 2.7.7.-...",4,1986-07-21,2018-01-31,2019-02-13,MGQTVTTPLSLTLEHWEDVQRIASNQSVDVKKRRWVTFCSAEWPTF...,1734
2,P02879,RICI_RICCO,True,False,1,3.2.2.22,4,1986-07-21,1987-08-13,2019-01-16,MKPGGNTIVIWMYAVATWLCFGSTSGWSFTLEDNNIFPKQYPIINF...,576


In [24]:
sprot2020.head(3)

,id,name,isenzyme,isMultiFunctional,functionCounts,ec_number,ec_specific_level,date_integraged,date_sequence_update,date_annotation_update,seq,seqlength
0,P03711,SCAF_LAMBD,True,False,1,3.4.21.-,3,1986-07-21,1986-07-21,2020-12-02,MTAELRNLPHIASMAFNEPLMLEPAYARVFFCALAGQLGISSLTDA...,439
1,P01027,CO3_MOUSE,False,False,0,-,0,1986-07-21,2011-07-27,2020-12-02,MGPASGSQLLVLLLLLASSPLALGIPMYSIITPNVLRLESEETIVL...,1663
2,P02706,ASGR1_RAT,False,False,0,-,0,1986-07-21,2007-01-23,2020-12-02,MTKDYQDFQHLDNENDHHQLQRGPPPAPRLLQRLCSGFRLFLLSLG...,284


In [25]:
sprot2021.head(3)

,id,name,isenzyme,isMultiFunctional,functionCounts,ec_number,ec_specific_level,date_integraged,date_sequence_update,date_annotation_update,seq,seqlength
0,P02802,MT1_MOUSE,False,False,0,-,0,1986-07-21,1986-07-21,2021-04-07,MDPNCSCSTGGSCTCTSSCACKNCKCTSCKKSCCSCCPVGCSKCAQ...,61
1,P02732,ANP3_PAGBO,False,False,0,-,0,1986-07-21,1986-07-21,2019-12-11,AATAATAATAATAATAATAATAATAATAATA,31
2,P02733,ANP3_PSEAM,False,False,0,-,0,1986-07-21,1986-07-21,2019-12-11,DTASDAAAAAALTAABAAAAAKLTABBAAAAAAATAA,37


In [26]:
sprot2022.head(3)

,id,name,isenzyme,isMultiFunctional,functionCounts,ec_number,ec_specific_level,date_integraged,date_sequence_update,date_annotation_update,seq,seqlength
0,P00250,FER_APHSA,False,False,0,-,0,1986-07-21,2007-01-23,2022-05-25,MASYKVTLKTPDGDNVITVPDDEYILDVAEEEGLDLPYSCRAGACS...,97
1,P03420,FUS_HRSVA,False,False,0,-,0,1986-07-21,1986-07-21,2022-05-25,MELLILKANAITTILTAVTFCFASGQNITEEFYQSTCSAVSKGYLS...,574
2,P0ACF7,DBHB_SHIFL,False,False,0,-,0,1986-07-21,1986-07-21,2022-05-25,MNKSQLIDKIAAGADISKAAAGRALDAIIASVTESLKEGDDVALVG...,90


## 5. Preprocessing
### 5.1 format EC

In [27]:
#sprot2018
sprot2018['ec_number'] = sprot2018.ec_number.parallel_apply(lambda x: mtool.format_ec(x))
sprot2018['ec_number'] = sprot2018.ec_number.parallel_apply(lambda x: mtool.specific_ecs(x))
sprot2018['functionCounts'] = sprot2018.ec_number.parallel_apply(lambda x: 0 if x=='-'  else len(x.split(',')))
print('sprot 2018 finished')

#sprot2019
sprot2019['ec_number'] = sprot2019.ec_number.parallel_apply(lambda x: mtool.format_ec(x))
sprot2019['ec_number'] = sprot2019.ec_number.parallel_apply(lambda x: mtool.specific_ecs(x))
sprot2019['functionCounts'] = sprot2019.ec_number.parallel_apply(lambda x: 0 if x=='-'  else len(x.split(',')))
print('sprot 2019 finished')

#sprot2020
sprot2020['ec_number'] = sprot2020.ec_number.parallel_apply(lambda x: mtool.format_ec(x))
sprot2020['ec_number'] = sprot2020.ec_number.parallel_apply(lambda x: mtool.specific_ecs(x))
sprot2020['functionCounts'] = sprot2020.ec_number.parallel_apply(lambda x: 0 if x=='-'  else len(x.split(',')))
print('sprot 2020 finished')

#sprot2021
sprot2021['ec_number'] = sprot2021.ec_number.parallel_apply(lambda x: mtool.format_ec(x))
sprot2021['ec_number'] = sprot2021.ec_number.parallel_apply(lambda x: mtool.specific_ecs(x))
sprot2021['functionCounts'] = sprot2021.ec_number.parallel_apply(lambda x: 0 if x=='-'  else len(x.split(',')))
print('sprot 2021 finished')


#sprot2022
sprot2022['ec_number'] = sprot2022.ec_number.parallel_apply(lambda x: mtool.format_ec(x))
sprot2022['ec_number'] = sprot2022.ec_number.parallel_apply(lambda x: mtool.specific_ecs(x))
sprot2022['functionCounts'] = sprot2022.ec_number.parallel_apply(lambda x: 0 if x=='-'  else len(x.split(',')))

print('sprot 2022 finished')

sprot 2018 finished
sprot 2019 finished
sprot 2020 finished
sprot 2021 finished
sprot 2022 finished


In [28]:
sprot2018.to_feather(cfg.DIR_UNIPROT + '/snap201802.feather')
sprot2019.to_feather(cfg.DIR_UNIPROT + '/snap201902.feather')
sprot2020.to_feather(cfg.DIR_UNIPROT + '/snap202006.feather')
sprot2021.to_feather(cfg.DIR_UNIPROT + '/snap202102.feather')
sprot2022.to_feather(cfg.DIR_UNIPROT + '/snap202202.feather')

### 5.2 Split Tain Test

In [16]:
sprot2018

,id,name,isenzyme,isMultiFunctional,functionCounts,ec_number,ec_specific_level,date_integraged,date_sequence_update,date_annotation_update,seq,seqlength
0,Q3J1A3,LHB1_RHOS4,False,False,0,-,0,1986-07-21,2007-01-23,2017-10-25,MADKSDLGYTGLTDEQAQELHSVYMSGLWLFSAVAIVAHLAVYIWRPWF,49
1,P02157,MYG_MELME,False,False,0,-,0,1986-07-21,2007-01-23,2017-11-22,MGLSDGEWQLVLNVWGKVEADLAGHGQEVLIRLFKGHPETLEKFDK...,154
2,P02178,MYG_MEGNO,False,False,0,-,0,1986-07-21,2007-01-23,2017-11-22,MVLSDAEWQLVLNIWAKVEADVAGHGQDILIRLFKGHPETLEKFDK...,154
3,P02194,MYG_MACRU,False,False,0,-,0,1986-07-21,2007-01-23,2017-11-22,MGLSDGEWQLVLNIWGKVETDEGGHGKDVLIRLFKGHPETLEKFDK...,154
4,P01915,HB22_MOUSE,False,False,0,-,0,1986-07-21,1986-07-21,2017-10-25,MVWLPRVPCVAAVILLLTVLSPPVALVRDTRPRFLEYVTSECHFYN...,264
...,...,...,...,...,...,...,...,...,...,...,...,...
469129,Q21221,AHO3_CAEEL,True,False,1,3.1.2.22,4,2018-02-28,2004-11-23,2018-02-28,MSSGAPSGSSMSSTPGSPPPRAGGPNSVSFKDLCCLFCCPPFPSSI...,332
469130,Q6QJ72,PDL2_ARATH,True,False,1,4.2.1.96,4,2018-02-28,2004-07-05,2018-02-28,MSRLLLPKLFSISRTQVPAASLFNNLYRRHKRFVHWTSKMSTDSVR...,187
469131,C0HL68,ES1GA_ODOGR,False,False,0,-,0,2018-02-28,2018-02-28,2018-02-28,GLFSKPAGKGIKNLIPKGVKHIGKEVGKDVIRTGIDVAGCKIKGEC,46
469132,C0HK74,VKT3_HETMG,False,False,0,-,0,2018-02-28,2018-02-28,2018-02-28,GSICLEPKVVGPCTAYFPRFYFDSETGKCTPFIYGGCEGNGNNFET...,56


In [29]:
train = sprot2018.iloc[:,np.r_[0,2:8,10:12]]

test_2019 = sprot2019.iloc[:,np.r_[0,2:8,10:12]]
test_2020 = sprot2020.iloc[:,np.r_[0,2:8,10:12]]
test_2021 = sprot2021.iloc[:,np.r_[0,2:8,10:12]]
test_2022 = sprot2022.iloc[:,np.r_[0,2:8,10:12]]

test_2019 =test_2019[~test_2019.seq.isin(train.seq)]
test_2019.reset_index(drop=True, inplace=True)

test_2020 =test_2020[~test_2020.seq.isin(train.seq)]
test_2020.reset_index(drop=True, inplace=True)

test_2021 =test_2021[~test_2021.seq.isin(train.seq)]
test_2021.reset_index(drop=True, inplace=True)

test_2022 =test_2022[~test_2022.seq.isin(train.seq)]
test_2022.reset_index(drop=True, inplace=True)

In [33]:
test_2019

,id,isenzyme,isMultiFunctional,functionCounts,ec_number,ec_specific_level,date_integraged,seq,seqlength
0,P02340,False,False,0,-,0,1986-07-21,MTAMEESQSDISLELPLSQETFSGLWKLLPPEDILPSPHCMDDLLL...,390
1,P01848,False,False,0,-,0,1986-07-21,IQNPDPAVYQLRDSKSSDKSVCLFTDFDSQTNVSQSKDSDVYITDK...,140
2,P01850,False,False,0,-,0,1986-07-21,DLNKVFPPEVAVFEPSEAEISHTQKATLVCLATGFFPDHVELSWWV...,176
3,P00939,True,True,2,"4.2.3.3,5.3.1.1",4,1986-07-21,MAGTGQEAEFRFSAFYISRQRPQPRPHGGTDLQCAGPSAMAPSRKF...,288
4,P01733,False,False,0,-,0,1986-07-21,MDSWTFCCVSLCILVAKHTDAGVIQSPRHEVTEMGQEVTLRCKPIS...,115
...,...,...,...,...,...,...,...,...,...
2877,Q5B8A2,True,False,1,2.5.1.-,3,2019-02-13,MYSKSWILNALPAPLVPYCELTRVGYLPIGVLVSYLPVLVAILHVA...,309
2878,Q5YTV5,True,False,1,1.14.13.-,3,2019-02-13,MIDVIIAGGGPTGLMLAGELRLHGVRTVVLEKEPTPNQHSRSRGLH...,473
2879,B4F779,False,False,0,-,0,2019-02-13,MPAVDKLLLEEALQDSPQTRSLLSVFEEDAGTLTDYTNQLLQAMQR...,662
2880,G4N287,True,False,1,1.-.-.-,1,2019-02-13,MKSFSLLASAGLATLASLPLTMAGVITPSYFDKHPLSRRQLSDAQV...,520


### 5.3 Remove changed seqence in test set

In [34]:
test_2019 = test_2019[~test_2019.id.isin(test_2019.merge(train, on='id', how='inner').id.values)]
test_2019.reset_index(drop=True, inplace=True)

test_2020 = test_2020[~test_2020.id.isin(test_2020.merge(train, on='id', how='inner').id.values)]
test_2020.reset_index(drop=True, inplace=True)

test_2021 = test_2021[~test_2021.id.isin(test_2021.merge(train, on='id', how='inner').id.values)]
test_2021.reset_index(drop=True, inplace=True)

test_2022 = test_2022[~test_2022.id.isin(test_2022.merge(train, on='id', how='inner').id.values)]
test_2022.reset_index(drop=True, inplace=True)


### 5.4 Trim string

In [35]:
with pd.option_context('mode.chained_assignment', None):
    train.ec_number = train.ec_number.parallel_apply(lambda x : str(x).strip()) #ec trim
    train.seq = train.seq.parallel_apply(lambda x : str(x).strip()) #seq trim

    test_2019.ec_number = test_2019.ec_number.parallel_apply(lambda x : str(x).strip()) #ec trim
    test_2019.seq = test_2019.seq.parallel_apply(lambda x : str(x).strip()) #seq trim
    
    test_2020.ec_number = test_2020.ec_number.parallel_apply(lambda x : str(x).strip()) #ec trim
    test_2020.seq = test_2020.seq.parallel_apply(lambda x : str(x).strip()) #seq trim

    test_2021.ec_number = test_2021.ec_number.parallel_apply(lambda x : str(x).strip()) #ec trim
    test_2021.seq = test_2021.seq.parallel_apply(lambda x : str(x).strip()) #seq trim

    test_2022.ec_number = test_2022.ec_number.parallel_apply(lambda x : str(x).strip()) #ec trim
    test_2022.seq = test_2022.seq.parallel_apply(lambda x : str(x).strip()) #seq trim

### 5.5 Save train test

In [36]:
train.to_feather(cfg.DATADIR + 'datasets/train.feather')
test_2019.to_feather(cfg.DATADIR + 'datasets/test_2019.feather')
test_2020.to_feather(cfg.DATADIR + 'datasets/test_2020.feather')
test_2021.to_feather(cfg.DATADIR + 'datasets/test_2021.feather')
test_2022.to_feather(cfg.DATADIR + 'datasets/test_2022.feather')

## 6. Build benchmarking datasets
### 6.1 Task 1 isEnzyme

In [37]:
train = pd.read_feather(cfg.DIR_DATASETS + 'train.feather')
test_2019 = pd.read_feather(cfg.DIR_DATASETS + 'test_2019.feather')
test_2020 = pd.read_feather(cfg.DIR_DATASETS + 'test_2020.feather')
test_2021 = pd.read_feather(cfg.DIR_DATASETS + 'test_2021.feather')
test_2022 = pd.read_feather(cfg.DIR_DATASETS + 'test_2022.feather')

task1_train = train.iloc[:,np.r_[0,7,1]]

task1_test_2019 = test_2019.iloc[:,np.r_[0,7,1]]
task1_test_2020 = test_2020.iloc[:,np.r_[0,7,1]]
task1_test_2021 = test_2021.iloc[:,np.r_[0,7,1]]
task1_test_2022 = test_2022.iloc[:,np.r_[0,7,1]]


task1_train.to_feather(cfg.FILE_TASK1_TRAIN)
task1_test_2019.to_feather(cfg.FILE_TASK1_TEST_2019)
task1_test_2020.to_feather(cfg.FILE_TASK1_TEST_2020)
task1_test_2021.to_feather(cfg.FILE_TASK1_TEST_2021)
task1_test_2022.to_feather(cfg.FILE_TASK1_TEST_2022)

funclib.table2fasta(table=task1_train[['id', 'seq']], file_out=cfg.FILE_TASK1_TRAIN_FASTA)
funclib.table2fasta(table=task1_test_2019[['id', 'seq']], file_out=cfg.FILE_TASK1_TEST_2019_FASTA)
funclib.table2fasta(table=task1_test_2020[['id', 'seq']], file_out=cfg.FILE_TASK1_TEST_2020_FASTA)
funclib.table2fasta(table=task1_test_2021[['id', 'seq']], file_out=cfg.FILE_TASK1_TEST_2021_FASTA)
funclib.table2fasta(table=task1_test_2022[['id', 'seq']], file_out=cfg.FILE_TASK1_TEST_2022_FASTA)

Write finished
Write finished
Write finished
Write finished
Write finished


### 6.2 Task2 Function Counts

In [38]:
task2_train = train[train.functionCounts >0]
task2_train.reset_index(drop=True, inplace=True)
task2_train = task2_train.iloc[:,np.r_[0,7,3]]

task2_test_2019 = test_2019[test_2019.functionCounts >0]
task2_test_2019.reset_index(drop=True, inplace=True)
task2_test_2019 = task2_test_2019.iloc[:,np.r_[0,7,3]]

task2_test_2020 = test_2020[test_2020.functionCounts >0]
task2_test_2020.reset_index(drop=True, inplace=True)
task2_test_2020 = task2_test_2020.iloc[:,np.r_[0,7,3]]

task2_test_2021 = test_2021[test_2021.functionCounts >0]
task2_test_2021.reset_index(drop=True, inplace=True)
task2_test_2021 = task2_test_2021.iloc[:,np.r_[0,7,3]]

task2_test_2022 = test_2022[test_2022.functionCounts >0]
task2_test_2022.reset_index(drop=True, inplace=True)
task2_test_2022 = task2_test_2022.iloc[:,np.r_[0,7,3]]

task2_train.to_feather(cfg.FILE_TASK2_TRAIN)
task2_test_2019.to_feather(cfg.FILE_TASK2_TEST_2019)
task2_test_2020.to_feather(cfg.FILE_TASK2_TEST_2020)
task2_test_2021.to_feather(cfg.FILE_TASK2_TEST_2021)
task2_test_2022.to_feather(cfg.FILE_TASK2_TEST_2022)

funclib.table2fasta(table=task2_train[['id', 'seq']], file_out=cfg.FILE_TASK2_TRAIN_FASTA)
funclib.table2fasta(table=task2_test_2019[['id', 'seq']], file_out=cfg.FILE_TASK2_TEST_2019_FASTA)
funclib.table2fasta(table=task2_test_2020[['id', 'seq']], file_out=cfg.FILE_TASK2_TEST_2020_FASTA)
funclib.table2fasta(table=task2_test_2021[['id', 'seq']], file_out=cfg.FILE_TASK2_TEST_2021_FASTA)
funclib.table2fasta(table=task2_test_2022[['id', 'seq']], file_out=cfg.FILE_TASK2_TEST_2022_FASTA)

Write finished
Write finished
Write finished
Write finished
Write finished


### 6.3 Task3 EC Number

In [39]:
task3_train = train[train.functionCounts >0]
task3_train.reset_index(drop=True, inplace=True)
task3_train = task3_train.iloc[:,np.r_[0,7,4]]

task3_test_2019 = test_2019[test_2019.functionCounts >0]
task3_test_2019.reset_index(drop=True, inplace=True)
task3_test_2019 = task3_test_2019.iloc[:,np.r_[0,7,4]]

task3_test_2020 = test_2020[test_2020.functionCounts >0]
task3_test_2020.reset_index(drop=True, inplace=True)
task3_test_2020 = task3_test_2020.iloc[:,np.r_[0,7,4]]

task3_test_2021 = test_2021[test_2021.functionCounts >0]
task3_test_2021.reset_index(drop=True, inplace=True)
task3_test_2021 = task3_test_2021.iloc[:,np.r_[0,7,4]]

task3_test_2022 = test_2022[test_2022.functionCounts >0]
task3_test_2022.reset_index(drop=True, inplace=True)
task3_test_2022 = task3_test_2022.iloc[:,np.r_[0,7,4]]

task3_train.to_feather(cfg.FILE_TASK3_TRAIN)
task3_test_2019.to_feather(cfg.FILE_TASK3_TEST_2019)
task3_test_2020.to_feather(cfg.FILE_TASK3_TEST_2020)
task3_test_2021.to_feather(cfg.FILE_TASK3_TEST_2021)
task3_test_2022.to_feather(cfg.FILE_TASK3_TEST_2022)

funclib.table2fasta(table=task3_train[['id', 'seq']], file_out=cfg.FILE_TASK3_TRAIN_FASTA)
funclib.table2fasta(table=task3_test_2019[['id', 'seq']], file_out=cfg.FILE_TASK3_TEST_2019_FASTA)
funclib.table2fasta(table=task3_test_2020[['id', 'seq']], file_out=cfg.FILE_TASK3_TEST_2020_FASTA)
funclib.table2fasta(table=task3_test_2021[['id', 'seq']], file_out=cfg.FILE_TASK3_TEST_2021_FASTA)
funclib.table2fasta(table=task3_test_2022[['id', 'seq']], file_out=cfg.FILE_TASK3_TEST_2022_FASTA)

Write finished
Write finished
Write finished
Write finished
Write finished


## 7 Make Feature Bank

### 7.1 ESM embedding 

In [12]:
# loading sprot data
snap18 = pd.read_feather(cfg.DIR_UNIPROT + '/snap201802.feather')
snap19 = pd.read_feather(cfg.DIR_UNIPROT + '/snap201902.feather')
snap20 = pd.read_feather(cfg.DIR_UNIPROT + '/snap202006.feather')
snap21 = pd.read_feather(cfg.DIR_UNIPROT + '/snap202102.feather')
snap22 = pd.read_feather(cfg.DIR_UNIPROT + '/snap202202.feather')
# merge
full_snap_data = pd.concat([snap18, snap19,snap20,snap21,snap22], axis=0)
full_snap_data = full_snap_data.sort_values(by=['id', 'date_annotation_update'], ascending=False)
full_snap_data = full_snap_data[['id', 'seq']].drop_duplicates(subset='id', keep='first')
full_snap_data.reset_index(drop=True, inplace=True)


# loading exsisting features
if ftool.isfileExists(cfg.FILE_FEATURE_ESM0):
    feature_esm0 = pd.read_feather(cfg.FILE_FEATURE_ESM0)
    feature_esm32 = pd.read_feather(cfg.FILE_FEATURE_ESM32)
    feature_esm33 = pd.read_feather(cfg.FILE_FEATURE_ESM33)
    feature_unirep = pd.read_feather(cfg.FILE_FEATURE_UNIREP)
    feature_onehot = pd.read_feather(cfg.FILE_FEATURE_ONEHOT)
    #caculate embedding list
    needesm = full_snap_data[~full_snap_data.id.isin(list(set(feature_esm33.id)))]
    needunirep = full_snap_data[~full_snap_data.id.isin(list(set(feature_unirep.id)))]
    needonehot = full_snap_data[~full_snap_data.id.isin(list(set(feature_onehot.id)))]
else:
    needesm = full_snap_data
    needunirep = full_snap_data
    needonehot = full_snap_data

In [3]:
# !pip install fair-esm
from tools import embedding_esm as esmebd
if len(needesm)>0:
    tr_rep0, tr_rep32, tr_rep33 = esmebd.get_rep_multi_sequence(sequences=needesm, model='esm1b_t33_650M_UR50S',seqthres=1022)

    #merge existing
    feature_esm0 = pd.concat([feature_esm0, tr_rep0], axis=0).reset_index(drop=True)
    feature_esm32 = pd.concat([feature_esm32, tr_rep32], axis=0).reset_index(drop=True)
    feature_esm33 = pd.concat([feature_esm33, tr_rep33], axis=0).reset_index(drop=True)


    #save
    feature_esm0.to_feather(cfg.FILE_FEATURE_ESM0)
    feature_esm32.to_feather(cfg.FILE_FEATURE_ESM32)
    feature_esm33.to_feather(cfg.FILE_FEATURE_ESM33)

Transferred model to GPU


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 35240/35240 [29:21<00:00, 20.00it/s]


### 7.2 Unirep

In [ ]:
if len(needunirep) > 0:
    from tools import embedding_unirep as unirep
    tr_unirep = unirep.getunirep(needunirep, 40)

    feature_unirep = pd.concat([feature_unirep, tr_unirep],axis=0).reset_index(drop=True)
    feature_unirep.to_feather(cfg.FILE_FEATURE_UNIREP)


### 7.3 one-hot

In [8]:
if len(needonehot) > 0:
    tr_onehot = onehotebd.get_onehot(sequences=needonehot, padding=True, padding_window=1500)
    feature_onehot = pd.concat([feature_onehot, tr_onehot],axis=0).reset_index(drop=True)
    feature_onehot.to_feather(cfg.FILE_FEATURE_ONEHOT)

In [10]:
feature_onehot

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f1491,f1492,f1493,f1494,f1495,f1496,f1497,f1498,f1499,f1500
0,X6R8R1,8193,128,2097152,2097152,2048,131072,32768,256,32768,...,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048
1,X6R8D5,8193,128,2,4096,32,256,32,131072,32768,...,4096,4096,4096,4096,4096,4096,4096,4096,4096,4096
2,X5M8U1,8193,2048,16384,2048,2,2048,16384,1024,16384,...,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048
3,X5M5N0,8193,32768,8,131072,1024,262144,4,128,128,...,1024,131072,32768,131072,4,32768,8,2097152,2048,262144
4,X5JB51,8193,262144,32,2,1024,2,1,2,128,...,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535956,Q71YA9,8193,256,1024,1,2048,1024,1,256,8,...,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024
535957,Q7MEP0,8193,64,4096,262144,262144,2,262144,8193,32768,...,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048
535958,A2RG75,8193,1024,262144,1024,32,64,2048,2048,8,...,1024,1024,1024,1024,1024,1024,1024,1024,1024,1024
535959,A7X0P6,8193,8,1,131072,262144,2048,16384,4096,4096,...,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048
